In [1]:
import sys
sys.path.append(r'D:\Users\60490\lh2')

import pandas as pd
from tqdm import tqdm

In [64]:
from lh.stockinfo.all_stock_daily import AllStock_daily
from lh.stockinfo.single_stock_daily import SingleStock_daily
from lh.time import Trade_time
asd = AllStock_daily()
ssd = SingleStock_daily()
tt = Trade_time()

daily_bans = pd.DataFrame(columns=['trade_date','ts_name','ban1','ban2','ban3','1to2','2to3','3still'])

date_beg = '20220101'
date_end = '20221231'
trade_dates = tt.getTradeDateList(date_beg, date_end)
ban1_pre, ban2_pre, ban3_pre = [], [], []
for trade_date in tqdm(trade_dates):
    ban1 = asd.get_only_k_bans_list(1, trade_date, True, True)
    ban2 = asd.get_only_k_bans_list(2, trade_date, True, True)
    ban3 = asd.get_least_k_bans_list(3, trade_date, True, True, True)
    ban1to2 = [1 if ts_name in ban1_pre else 0 for ts_name in ban2]
    ban2to3 = [1 if ts_name in ban2_pre else 0 for ts_name in ban3]
    ban3still = [1 if ts_name in ban3_pre else 0 for ts_name in ban3]
    # print(trade_date)
    # print(ban1,ban2,ban3,sep='\n')
    daily_bans = pd.concat([daily_bans, 
                            pd.DataFrame({'trade_date':trade_date,'ts_name':ban1,
                                          'ban1':1,'ban2':0,'ban3':0,
                                          '1to2':0,'2to3':0,'3still':0})],ignore_index=True)
    daily_bans = pd.concat([daily_bans, 
                            pd.DataFrame({'trade_date':trade_date,'ts_name':ban2,
                                          'ban1':0,'ban2':1,'ban3':0,
                                          '1to2':ban1to2,'2to3':0,'3still':0})],ignore_index=True)
    daily_bans = pd.concat([daily_bans, 
                            pd.DataFrame({'trade_date':trade_date,'ts_name':ban3,
                                          'ban1':0,'ban2':0,'ban3':1,
                                          '1to2':0,'2to3':ban2to3,'3still':ban3still})],ignore_index=True)
    ban1_pre, ban2_pre, ban3_pre = ban1, ban2, ban3

print(daily_bans)

100%|██████████| 243/243 [29:10<00:00,  7.20s/it]

      trade_date ts_name ban1 ban2 ban3 1to2 2to3 3still
0       20220101    柳药集团    1    0    0    0    0      0
1       20220101    成飞集成    1    0    0    0    0      0
2       20220101    亚钾国际    1    0    0    0    0      0
3       20220101    开开实业    1    0    0    0    0      0
4       20220101    东北制药    1    0    0    0    0      0
...          ...     ...  ...  ...  ...  ...  ...    ...
17244   20221230    中兴商业    0    0    1    0    0      1
17245   20221230    安妮股份    0    0    1    0    1      0
17246   20221230    中科云网    0    0    1    0    1      0
17247   20221230     桂发祥    0    0    1    0    1      0
17248   20221230    尚太科技    0    0    1    0    1      0

[17249 rows x 8 columns]


In [65]:
with open(f'daily_bans_{date_beg}-{date_end}.csv', 'w', encoding='utf8') as fout:
    daily_bans.to_csv(fout)

In [66]:
daily_bans_sumup = daily_bans.groupby('trade_date').sum()[['ban1','ban2','ban3','2to3','3still']]
print(daily_bans_sumup)
with open(f'daily_bans_sumup_{date_beg}-{date_end}.csv', 'w', encoding='utf8') as fout:
    daily_bans_sumup.to_csv(fout)

            ban1  ban2  ban3  2to3  3still
trade_date                                
20220101      42    10    14     0       0
20220104      78    20    13     5       8
20220105      30    19    13     6       7
20220106      70     8    11     6       5
20220107      28     7    10     2       8
...          ...   ...   ...   ...     ...
20221226      59     2     5     4       1
20221227      36    13     4     1       3
20221228      24     4     9     7       2
20221229      35     6     7     2       5
20221230      40     6     6     4       2

[243 rows x 5 columns]


### 二板明日有溢价的概率

In [5]:
from lh.stockinfo.all_stock_daily import AllStock_daily
from lh.stockinfo.single_stock_daily import SingleStock_daily
from lh.time import Trade_time
asd = AllStock_daily()
ssd = SingleStock_daily()
tt = Trade_time()

date_beg = '20220901'
date_end = '20230401'
trade_dates = tt.getTradeDateList(date_beg, date_end)
ban2_pre = asd.get_only_k_bans_list(2, tt.preTradeDate(trade_dates[0]), False, True)
cnt_ban2 = len(ban2_pre)
cnt_ban2_yijia = 0
for trade_date in tqdm(trade_dates):
    ban2 = asd.get_only_k_bans_list(2, trade_date, False, True)
    cnt_ban2 += len(ban2)
    
    for ts_code in ban2_pre:
        daily = ssd.getDaily_df(ts_code, trade_date)
        cnt_ban2_yijia += 1 if daily.pre_close.iloc[0] < daily.high.iloc[0] else 0

    ban2_pre = ban2


print(cnt_ban2_yijia, cnt_ban2, cnt_ban2_yijia/cnt_ban2)

100%|██████████| 140/140 [08:29<00:00,  3.64s/it]

829 904 0.9170353982300885


In [6]:
from lh.stockinfo.all_stock_daily import AllStock_daily
from lh.stockinfo.single_stock_daily import SingleStock_daily
from lh.time import Trade_time
asd = AllStock_daily()
ssd = SingleStock_daily()
tt = Trade_time()

date_beg = '20220901'
date_end = '20230401'
trade_dates = tt.getTradeDateList(date_beg, date_end)
MAX_K = 9
bank_pres = [asd.get_only_k_bans_list(k, tt.preTradeDate(trade_dates[0]), False, True) for k in range(1,MAX_K+1)]
cnt_banks = [len(bank_pre) for bank_pre in bank_pres]
cnt_bank_yijias = [0]*MAX_K
for trade_date in tqdm(trade_dates):
    banks = [asd.get_only_k_bans_list(k, trade_date, False, True) for k in range(1,MAX_K+1)]
    for i in range(MAX_K):
        cnt_banks[i] += len(banks[i])
    
    for i in range(MAX_K):
        for ts_code in bank_pres[i]:
            daily = ssd.getDaily_df(ts_code, trade_date)
            cnt_bank_yijias[i] += 1 if daily.pre_close.iloc[0] < daily.high.iloc[0] else 0

    bank_pres = banks

for i in range(MAX_K):
    k = i+1
    print(k, cnt_bank_yijias[i], cnt_banks[i], cnt_bank_yijias[i]/cnt_banks[i])

100%|██████████| 140/140 [2:34:50<00:00, 66.36s/it] 

1 4513 5086 0.8873377900117971
2 829 904 0.9170353982300885
3 320 360 0.8888888888888888
4 155 164 0.9451219512195121
5 82 96 0.8541666666666666
6 40 47 0.851063829787234
7 28 29 0.9655172413793104
8 13 14 0.9285714285714286
9 8 9 0.8888888888888888


### 二板第二天开盘入第三天有溢价的概率

In [8]:
from lh.stockinfo.all_stock_daily import AllStock_daily
from lh.stockinfo.single_stock_daily import SingleStock_daily
from lh.time import Trade_time
asd = AllStock_daily()
ssd = SingleStock_daily()
tt = Trade_time()

date_beg = '20220901'
date_end = '20230401'
trade_dates = tt.getTradeDateList(date_beg, date_end)
ban2_pre = asd.get_only_k_bans_list(2, tt.preTradeDate(trade_dates[0]), False, True)
ban2_post = []
cnt_ban2_post = 0
cnt_ban2_post_yijia = 0
for trade_date in tqdm(trade_dates):
    ban2 = asd.get_only_k_bans_list(2, trade_date, False, True)

    for ts_code,buy_price in ban2_post:
        daily = ssd.getDaily_df(ts_code, trade_date)
        cnt_ban2_post_yijia += 1 if daily.high.iloc[0] > buy_price else 0
    
    ban2_post = []
    for ts_code in ban2_pre:
        daily = ssd.getDaily_df(ts_code, trade_date)
        if daily.low.iloc[0] == daily.high.iloc[0]:
            continue
        cnt_ban2_post += 1
        ban2_post.append((ts_code,daily.open.iloc[0]))

    ban2_pre = ban2


print(cnt_ban2_post, cnt_ban2_post_yijia, cnt_ban2_post_yijia/cnt_ban2_post)

100%|██████████| 140/140 [10:14<00:00,  4.39s/it]

797 506 0.6348808030112923


In [17]:
from lh.stockinfo.all_stock_daily import AllStock_daily
from lh.stockinfo.single_stock_daily import SingleStock_daily
from lh.time import Trade_time
asd = AllStock_daily()
ssd = SingleStock_daily()
tt = Trade_time()

date_beg = '20220901'
date_end = '20230401'
trade_dates = tt.getTradeDateList(date_beg, date_end)
MAX_K = 9
bank_pres = [asd.get_only_k_bans_list(k, tt.preTradeDate(trade_dates[0]), False, True) for k in range(1,MAX_K+1)]
bank_posts = [[]]*MAX_K
cnt_bank_posts = [0 for _ in range(MAX_K)]
cnt_bank_post_yijias = [0]*MAX_K
for trade_date in tqdm(trade_dates):
    banks = [asd.get_only_k_bans_list(k, trade_date, False, True) for k in range(1,MAX_K+1)]

    for i in range(MAX_K):
        for ts_code,buy_price in bank_posts[i]:
            daily = ssd.getDaily_df(ts_code, trade_date)
            cnt_bank_post_yijias[i] += 1 if daily.high.iloc[0] > buy_price else 0
    
    # bank_posts = [[]]*MAX_K  # list嵌套这么写的话内部多个list其实都指向同一个对象
    bank_posts = [[] for _ in range(MAX_K)]
    for i in range(MAX_K):
        for ts_code in bank_pres[i]:
            daily = ssd.getDaily_df(ts_code, trade_date)
            if daily.low.iloc[0] == daily.high.iloc[0] or daily.vol.iloc[0]==0:
                continue
            cnt_bank_posts[i] += 1
            bank_posts[i].append((ts_code,daily.open.iloc[0]))

    bank_pres = banks


print(cnt_ban2_post, cnt_ban2_post_yijia, cnt_ban2_post_yijia/cnt_ban2_post)
for i in range(MAX_K):
    k = i+1
    print(k, cnt_bank_posts[i], cnt_bank_post_yijias[i], cnt_bank_post_yijias[i]/cnt_bank_posts[i])

100%|██████████| 140/140 [2:51:41<00:00, 73.58s/it] 

797 506 0.6348808030112923
1 4834 3093 0.6398427803061647
2 797 506 0.6348808030112923
3 297 182 0.6127946127946128
4 124 84 0.6774193548387096
5 78 43 0.5512820512820513
6 33 24 0.7272727272727273
7 23 11 0.4782608695652174
8 10 8 0.8
9 4 2 0.5
